In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df1 = pd.read_json('project_data/schoolInfo.json')

In [4]:
df1.head()

,rankingNoteText,nonResponderText,nonResponder,act-avg,primaryPhoto,primaryPhotoThumb,sat-avg,enrollment,city,sortName,...,displayName,schoolType,region,aliasNames,rankingType,overallRank,institutionalControl,rankingMaxPossibleScore,rankingRankStatus,primaryKey
0,NaN,None,False,32.0,https://www.usnews.com/img/college-photo_31291...,https://www.usnews.com/img/college-photo_31291...,1400.0,5400.0,Princeton,princetonuniversity,...,Princeton University,national-universities,NaN,None,national-universities,1,private,NaN,ranked,2627
1,NaN,None,False,32.0,https://www.usnews.com/img/college-photo_8866.jpg,https://www.usnews.com/img/college-photo_8866_...,1430.0,6710.0,Cambridge,harvarduniversity,...,Harvard University,national-universities,NaN,None,national-universities,2,private,NaN,ranked,2155
2,NaN,None,False,32.0,https://www.usnews.com/dims4/USNEWS/5b128f0/17...,https://www.usnews.com/dims4/USNEWS/196469f/17...,1450.0,5941.0,Chicago,universityofchicago,...,University of Chicago,national-universities,NaN,UChicago,national-universities,3,private,NaN,ranked,1774
3,NaN,None,False,32.0,https://www.usnews.com/dims4/USNEWS/60348dd/17...,https://www.usnews.com/dims4/USNEWS/1ca230f/17...,1420.0,5472.0,New Haven,yaleuniversity,...,Yale University,national-universities,NaN,None,national-universities,3,private,NaN,ranked,1426
4,NaN,None,False,32.0,https://www.usnews.com/img/college-photo_19002...,https://www.usnews.com/img/college-photo_19002...,1430.0,6113.0,New York,columbiauniversity,...,Columbia University,national-universities,NaN,None,national-universities,5,private,NaN,ranked,2707


In [5]:
df1.isna().sum()

rankingNoteText            311
nonResponderText           300
nonResponder                 0
act-avg                     20
primaryPhoto                47
primaryPhotoThumb           47
sat-avg                     20
enrollment                  11
city                         0
sortName                     0
zip                          0
rankingNoteCharacter       311
acceptance-rate              9
rankingDisplayScore         81
percent-receiving-aid      168
cost-after-aid             168
state                        0
rankingSortRank              0
hs-gpa-avg                  67
urlName                      0
rankingDisplayName           0
rankingDisplayRank           0
ranking                      0
xwalkId                      0
rankingIsTied                0
isPublic                     0
businessRepScore            77
tuition                      0
engineeringRepScore        105
displayName                  0
schoolType                   0
region                     311
aliasNam

In [6]:
# Drop columns which only have NaN values
df1.dropna(axis=1, how='all', inplace=True)

In [7]:
df1.shape

(311, 35)

In [8]:
df1.isna().sum()

nonResponderText         300
nonResponder               0
act-avg                   20
primaryPhoto              47
primaryPhotoThumb         47
sat-avg                   20
enrollment                11
city                       0
sortName                   0
zip                        0
acceptance-rate            9
rankingDisplayScore       81
percent-receiving-aid    168
cost-after-aid           168
state                      0
rankingSortRank            0
hs-gpa-avg                67
urlName                    0
rankingDisplayName         0
rankingDisplayRank         0
ranking                    0
xwalkId                    0
rankingIsTied              0
isPublic                   0
businessRepScore          77
tuition                    0
engineeringRepScore      105
displayName                0
schoolType                 0
aliasNames               167
rankingType                0
overallRank                0
institutionalControl       0
rankingRankStatus          0
primaryKey    

In [9]:
# Drop irrelevant variables
df1 = df1.drop(['nonResponderText', 'nonResponder', 'primaryPhoto', 'primaryPhotoThumb', 'aliasNames', 'urlName'], 
         axis = 1)
df1.shape

(311, 29)

In [11]:
df1.isna().sum()

act-avg                   20
sat-avg                   20
enrollment                11
city                       0
sortName                   0
zip                        0
acceptance-rate            9
rankingDisplayScore       81
percent-receiving-aid    168
cost-after-aid           168
state                      0
rankingSortRank            0
hs-gpa-avg                67
rankingDisplayName         0
rankingDisplayRank         0
ranking                    0
xwalkId                    0
rankingIsTied              0
isPublic                   0
businessRepScore          77
tuition                    0
engineeringRepScore      105
displayName                0
schoolType                 0
rankingType                0
overallRank                0
institutionalControl       0
rankingRankStatus          0
primaryKey                 0
dtype: int64

In [93]:
missing_rows = df1[df1['enrollment'].isna()]
missing_rows.head()

,act-avg,sat-avg,enrollment,city,sortName,zip,acceptance-rate,rankingDisplayScore,percent-receiving-aid,cost-after-aid,...,businessRepScore,tuition,engineeringRepScore,displayName,schoolType,rankingType,overallRank,institutionalControl,rankingRankStatus,primaryKey
241,NaN,NaN,NaN,Phoenix,grandcanyonuniversity,85017,43.0,NaN,NaN,NaN,...,NaN,17050,NaN,Grand Canyon University,national-universities,national-universities,-1,proprietary,rnp,1074
251,NaN,NaN,NaN,Chicago,nationallouisuniversity,60603,81.0,NaN,NaN,NaN,...,NaN,10380,NaN,National Louis University,national-universities,national-universities,-1,private,rnp,1733
262,18.0,NaN,NaN,Louisville,spaldinguniversity,40203,50.0,NaN,NaN,NaN,...,NaN,24388,NaN,Spalding University,national-universities,national-universities,-1,private,rnp,1960
301,NaN,NaN,NaN,Orange,argosyuniversity,92868,NaN,NaN,NaN,NaN,...,NaN,13152,NaN,Argosy University,national-universities,national-universities,-2,proprietary,ur,21799
302,NaN,NaN,NaN,San Francisco,californiainstituteofintegralstudies,94103,NaN,NaN,NaN,NaN,...,NaN,28467,NaN,California Institute of Integral Studies,national-universities,national-universities,-2,private,ur,12154


In [92]:
df2 = pd.read_csv('project_data/MERGED2017_18_PP.csv')
df2.head()

/var/folders/vk/7c21x7qj4fv7flh8p9m2wwn00000gn/T/ipykernel_1671/1963960516.py:1: DtypeWarning: Columns (1729,1909,1910,1911,1912,1913) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('project_data/MERGED2017_18_PP.csv')


,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,COUNT_WNE_MALE1_P8,MD_EARN_WNE_MALE1_P8,GT_THRESHOLD_P10,MD_EARN_WNE_INC1_P10,MD_EARN_WNE_INC2_P10,MD_EARN_WNE_INC3_P10,MD_EARN_WNE_INDEP1_P10,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
df2.shape

(7112, 2989)

In [95]:
#Manually selected some variables that seem relevant based on description
df2_slice = df2[['OPEID6','INSTNM','SCH_DEG','NUMBRANCH','PREDDEG','HIGHDEG','REGION','ADM_RATE','SATVR25','SATVR75','SATMT25',
                'SATMT75','SATWR25','SATWR75','SATVRMID','SATMTMID','SATWRMID','ACTCM25','ACTCM75','ACTEN25',
                'ACTEN75','ACTMT25','ACTMT75','ACTWR25','ACTWR75','ACTCMMID','ACTENMID','ACTMTMID','ACTWRMID',
                'SAT_AVG','UGDS','UGDS_WHITE','UGDS_BLACK','UGDS_HISP','UGDS_ASIAN','UGDS_AIAN','UGDS_NHPI',
                'UGDS_2MOR','UGDS_NRA','UGDS_UNKN','PPTUG_EF','NPT4_PUB','NPT4_PRIV','NUM4_PUB','NUM4_PRIV',
                'NUM4_PROG','NUM4_OTHER','COSTT4_A','COSTT4_P','TUITIONFEE_IN','TUITIONFEE_OUT','TUITIONFEE_PROG',
                'TUITFTE','INEXPFTE','AVGFACSAL','PFTFAC']]

In [96]:
df2_slice.shape

(7112, 56)

In [97]:
df2_slice.isna().sum()

OPEID6                0
INSTNM                0
SCH_DEG             132
NUMBRANCH             0
PREDDEG               0
HIGHDEG               0
REGION                0
ADM_RATE           5075
SATVR25            5874
SATVR75            5874
SATMT25            5874
SATMT75            5874
SATWR25            7112
SATWR75            7112
SATVRMID           5874
SATMTMID           5874
SATWRMID           7112
ACTCM25            5817
ACTCM75            5817
ACTEN25            5897
ACTEN75            5897
ACTMT25            5897
ACTMT75            5897
ACTWR25            7112
ACTWR75            7112
ACTCMMID           5817
ACTENMID           5897
ACTMTMID           5897
ACTWRMID           7112
SAT_AVG            5792
UGDS                748
UGDS_WHITE          749
UGDS_BLACK          749
UGDS_HISP           749
UGDS_ASIAN          749
UGDS_AIAN           749
UGDS_NHPI           749
UGDS_2MOR           749
UGDS_NRA            749
UGDS_UNKN           749
PPTUG_EF            765
NPT4_PUB        

In [98]:
#Drop columns with only NA
df2_slice.dropna(axis=1, how='all', inplace=True)

/var/folders/vk/7c21x7qj4fv7flh8p9m2wwn00000gn/T/ipykernel_1671/1375013343.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_slice.dropna(axis=1, how='all', inplace=True)


In [99]:
df2_slice.shape

(7112, 48)

In [100]:
df2_slice['primaryKey'] = df2_slice['OPEID6']

/var/folders/vk/7c21x7qj4fv7flh8p9m2wwn00000gn/T/ipykernel_1671/3893990605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_slice['primaryKey'] = df2_slice['OPEID6']


In [101]:
result = pd.merge(df1, df2_slice, on='primaryKey', how="left", indicator = True)
result.shape

(420, 78)

In [73]:
result.head()

,act-avg,sat-avg,enrollment,city,sortName,zip,acceptance-rate,rankingDisplayScore,percent-receiving-aid,cost-after-aid,...,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PFTFAC,_merge
0,32.0,1400.0,5400.0,Princeton,princetonuniversity,8544,7.0,100.0,60.0,16793.0,...,64390.0,NaN,47140.0,47140.0,NaN,13216.0,53658.0,17519.0,0.8490,both
1,32.0,1430.0,6710.0,Cambridge,harvarduniversity,2138,5.0,98.0,55.0,16338.0,...,66900.0,NaN,48949.0,48949.0,NaN,31930.0,43823.0,19741.0,0.8589,both
2,32.0,1450.0,5941.0,Chicago,universityofchicago,60637,8.0,96.0,42.0,27767.0,...,72717.0,NaN,56034.0,56034.0,NaN,28508.0,89151.0,18115.0,0.8239,both
3,32.0,1420.0,5472.0,New Haven,yaleuniversity,6520,6.0,96.0,50.0,18385.0,...,68950.0,NaN,51400.0,51400.0,NaN,17934.0,120956.0,17517.0,0.7459,both
4,32.0,1430.0,6113.0,New York,columbiauniversity,10027,6.0,95.0,48.0,21041.0,...,71972.0,NaN,57208.0,57208.0,NaN,37294.0,89742.0,18713.0,0.4688,both


In [102]:
duplicated = result[result['primaryKey'].duplicated(keep=False)]
duplicated.shape

(148, 78)

In [79]:
result.to_csv('processed_data1.csv', index=False)

In [80]:
duplicated.to_csv('duplicated.csv', index=False)

In [62]:
df1.to_csv('kaggle_data.csv',index=False)

In [161]:
#Processed some data manually
data = pd.read_csv('project_data/processed_data1.csv')
data.head()

,act-avg,sat-avg,enrollment,city,sortName,zip,acceptance-rate,rankingDisplayScore,percent-receiving-aid,cost-after-aid,...,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PFTFAC,_merge
0,32.0,1400.0,5400.0,Princeton,princetonuniversity,8544,7.0,100.0,60.0,16793.0,...,64390.0,NaN,47140.0,47140.0,NaN,13216.0,53658.0,17519.0,0.8490,both
1,32.0,1430.0,6710.0,Cambridge,harvarduniversity,2138,5.0,98.0,55.0,16338.0,...,66900.0,NaN,48949.0,48949.0,NaN,31930.0,43823.0,19741.0,0.8589,both
2,32.0,1450.0,5941.0,Chicago,universityofchicago,60637,8.0,96.0,42.0,27767.0,...,72717.0,NaN,56034.0,56034.0,NaN,28508.0,89151.0,18115.0,0.8239,both
3,32.0,1420.0,5472.0,New Haven,yaleuniversity,6520,6.0,96.0,50.0,18385.0,...,68950.0,NaN,51400.0,51400.0,NaN,17934.0,120956.0,17517.0,0.7459,both
4,32.0,1430.0,6113.0,New York,columbiauniversity,10027,6.0,95.0,48.0,21041.0,...,71972.0,NaN,57208.0,57208.0,NaN,37294.0,89742.0,18713.0,0.4688,both


In [162]:
data.shape

(311, 78)

In [163]:
#Identify unranked institutions
data.loc[data.rankingDisplayRank == "Unranked"]

,act-avg,sat-avg,enrollment,city,sortName,zip,acceptance-rate,rankingDisplayScore,percent-receiving-aid,cost-after-aid,...,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PFTFAC,_merge
300,NaN,NaN,1264.0,San Diego,alliantinternationaluniversity,92131,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
301,NaN,NaN,NaN,Orange,argosyuniversity,92868,NaN,NaN,NaN,NaN,...,29396.0,NaN,13438.0,13438.0,NaN,16793.0,6133.0,5460.0,0.1774,both
302,NaN,NaN,NaN,San Francisco,californiainstituteofintegralstudies,94103,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,22916.0,13260.0,8190.0,0.3247,both
303,NaN,NaN,NaN,Minneapolis,capellauniversity,55403,NaN,NaN,NaN,NaN,...,19836.0,NaN,14250.0,14250.0,NaN,16533.0,1714.0,6841.0,0.1387,both
304,NaN,NaN,NaN,Pocatello,idahostateuniversity,83209,NaN,NaN,NaN,NaN,...,19592.0,NaN,7166.0,21942.0,NaN,7822.0,11741.0,7156.0,0.9380,both
305,NaN,NaN,133.0,San Diego,northcentraluniversity,86314,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16529.0,2909.0,6347.0,0.2204,left_only
306,NaN,NaN,NaN,Cypress,tridentuniversityinternational,90630,96.0,NaN,NaN,NaN,...,17544.0,NaN,9240.0,9240.0,NaN,9361.0,1828.0,5705.0,0.0614,both
307,NaN,NaN,NaN,Cincinnati,unioninstituteanduniversity,45206,NaN,NaN,NaN,NaN,...,24696.0,NaN,12896.0,12896.0,NaN,16910.0,6638.0,5540.0,0.1690,both
308,NaN,NaN,NaN,Phoenix,universityofphoenix,85034,NaN,NaN,NaN,NaN,...,20083.0,NaN,9608.0,9608.0,NaN,13180.0,2042.0,4485.0,0.0462,left_only
309,NaN,NaN,NaN,Minneapolis,waldenuniversity,55401,NaN,NaN,NaN,NaN,...,NaN,NaN,12465.0,12465.0,NaN,10183.0,2854.0,6769.0,0.0674,both


In [164]:
#Remove unranked institutions
data.drop(index = range(300,311), inplace = True)
data.shape

(300, 78)

In [165]:
#drop columns with only NA
data.dropna(axis=1, how='all', inplace=True)
data.shape

(300, 76)

In [166]:
#Combine columns that have the same information but stored separately for public and private institutions
data['NPT4'] = data['NPT4_PUB'].fillna(data['NPT4_PRIV'])
data['NUM4'] = data['NUM4_PUB'].fillna(data['NUM4_PRIV'])

In [167]:
#Drop already combined columns and _merge
data.drop(['_merge','NPT4_PUB','NPT4_PRIV','NUM4_PUB','NUM4_PRIV'], axis=1, inplace=True)

In [168]:
data.shape

(300, 73)

In [169]:
#Drop columns with only 1 unique value
cols_to_drop = []
for col in data.columns:
    if data[col].nunique() == 1:
        cols_to_drop.append(col)
data.drop(cols_to_drop, axis=1, inplace = True)

In [170]:
#Look at how many unique values other columns have
unique_values_dict = {}
for col in data.columns:
    unique_values_dict[col] = data[col].nunique()
print(unique_values_dict)

{'act-avg': 20, 'sat-avg': 79, 'enrollment': 296, 'city': 236, 'sortName': 300, 'zip': 296, 'acceptance-rate': 86, 'rankingDisplayScore': 60, 'percent-receiving-aid': 59, 'cost-after-aid': 143, 'state': 51, 'rankingSortRank': 61, 'hs-gpa-avg': 12, 'rankingDisplayRank': 61, 'xwalkId': 300, 'rankingIsTied': 2, 'businessRepScore': 26, 'tuition': 299, 'engineeringRepScore': 29, 'displayName': 300, 'overallRank': 61, 'institutionalControl': 3, 'rankingRankStatus': 2, 'primaryKey': 300, 'OPEID6': 298, 'INSTNM': 299, 'NUMBRANCH': 9, 'REGION': 8, 'ADM_RATE': 293, 'SATVR25': 45, 'SATVR75': 38, 'SATMT25': 45, 'SATMT75': 39, 'SATVRMID': 70, 'SATMTMID': 73, 'ACTCM25': 20, 'ACTCM75': 17, 'ACTEN25': 22, 'ACTEN75': 18, 'ACTMT25': 20, 'ACTMT75': 18, 'ACTCMMID': 19, 'ACTENMID': 20, 'ACTMTMID': 20, 'SAT_AVG': 205, 'UGDS': 299, 'UGDS_WHITE': 293, 'UGDS_BLACK': 278, 'UGDS_HISP': 277, 'UGDS_ASIAN': 276, 'UGDS_AIAN': 88, 'UGDS_NHPI': 55, 'UGDS_2MOR': 215, 'UGDS_NRA': 274, 'UGDS_UNKN': 228, 'PPTUG_EF': 275, 

In [171]:
data['sat-avg'].isna().sum()

9

In [172]:
data['SAT_AVG'].isna().sum()

12

In [173]:
missing = data['sat-avg'].isna()
na_rows = data[missing]
na_rows

,act-avg,sat-avg,enrollment,city,sortName,zip,acceptance-rate,rankingDisplayScore,percent-receiving-aid,cost-after-aid,...,PPTUG_EF,COSTT4_A,TUITIONFEE_IN,TUITIONFEE_OUT,TUITFTE,INEXPFTE,AVGFACSAL,PFTFAC,NPT4,NUM4
117,23.0,NaN,19262.0,Lawrence,universityofkansas,66045,93.0,47.0,30.0,30962.0,...,0.0982,24824.0,10824.0,26592.0,12206.0,18034.0,9976.0,0.8282,18814.0,1426.0
159,21.0,NaN,18643.0,Normal,illinoisstateuniversity,61790,89.0,38.0,NaN,NaN,...,0.0616,28197.0,14061.0,25168.0,11552.0,15761.0,7979.0,0.7675,21345.0,2496.0
214,19.0,NaN,18313.0,Kalamazoo,westernmichiganuniversity,49008,82.0,29.0,NaN,NaN,...,0.1519,24357.0,11943.0,14699.0,12351.0,11914.0,9751.0,0.6262,17259.0,1850.0
223,20.0,NaN,3171.0,Lisle,benedictineuniversity,60532,73.0,27.0,NaN,NaN,...,0.1393,42641.0,33900.0,33900.0,15350.0,6919.0,8210.0,0.2576,23827.0,448.0
241,NaN,NaN,NaN,Phoenix,grandcanyonuniversity,85017,43.0,NaN,NaN,NaN,...,0.6716,31051.0,17050.0,17050.0,10977.0,1997.0,6482.0,0.0709,20570.0,1775.0
251,NaN,NaN,NaN,Chicago,nationallouisuniversity,60603,81.0,NaN,NaN,NaN,...,0.3083,24158.0,10440.0,10440.0,15427.0,6151.0,6148.0,0.9866,15579.0,283.0
254,19.0,NaN,14079.0,DeKalb,northernillinoisuniversity,60115,52.0,NaN,NaN,NaN,...,0.1224,28557.0,14350.0,23816.0,7816.0,14104.0,8218.0,0.7511,19823.0,1359.0
262,18.0,NaN,NaN,Louisville,spaldinguniversity,40203,50.0,NaN,NaN,NaN,...,0.1519,33455.0,24000.0,24000.0,17087.0,8211.0,7326.0,0.4405,21872.0,149.0
296,20.0,NaN,21375.0,Milwaukee,universityofwisconsinmilwaukee,53201,72.0,NaN,NaN,NaN,...,0.1338,21701.0,9565.0,20844.0,8765.0,8925.0,8426.0,0.6545,16650.0,1951.0


In [174]:
missing = data['SAT_AVG'].isna()
na_rows = data[missing]
na_rows

,act-avg,sat-avg,enrollment,city,sortName,zip,acceptance-rate,rankingDisplayScore,percent-receiving-aid,cost-after-aid,...,PPTUG_EF,COSTT4_A,TUITIONFEE_IN,TUITIONFEE_OUT,TUITFTE,INEXPFTE,AVGFACSAL,PFTFAC,NPT4,NUM4
26,28.0,1240.0,4955.0,Winston-Salem,wakeforestuniversity,27109,30.0,75.0,30.0,24241.0,...,0.0108,66440.0,51400.0,51400.0,32036.0,28205.0,11795.0,0.7714,33382.0,397.0
65,28.0,1200.0,4432.0,Worcester,worcesterpolytechnicinstitute,1609,48.0,60.0,62.0,39995.0,...,0.0214,62522.0,48628.0,48628.0,26622.0,19378.0,11971.0,0.8416,40376.0,659.0
80,27.0,1110.0,2289.0,Worcester,clarkuniversity,1610,55.0,54.0,62.0,27702.0,...,0.0082,53086.0,44400.0,44400.0,18428.0,12561.0,10701.0,1.0000,27004.0,316.0
116,24.0,1050.0,29275.0,Philadelphia,templeuniversity,19122,52.0,47.0,37.0,35109.0,...,0.0862,31599.0,16658.0,28418.0,22122.0,13574.0,10798.0,0.5564,23366.0,2595.0
119,23.0,1040.0,3241.0,Washington,catholicuniversityofamerica,20064,80.0,46.0,54.0,35518.0,...,0.0292,59591.0,44060.0,44060.0,23131.0,19118.0,9645.0,0.5298,32731.0,460.0
120,22.0,1030.0,15407.0,Chicago,depauluniversity,60604,70.0,46.0,59.0,42438.0,...,0.1186,52969.0,39010.0,39010.0,23156.0,12582.0,10699.0,0.4938,30559.0,1588.0
127,21.0,970.0,34072.0,Tucson,universityofarizona,85721,79.0,45.0,29.0,35051.0,...,0.1250,26157.0,11877.0,35307.0,15765.0,12211.0,10929.0,0.7484,14821.0,1958.0
144,22.0,950.0,19472.0,Manhattan,kansasstateuniversity,66506,94.0,41.0,22.0,34621.0,...,0.0836,23262.0,10135.0,25492.0,11305.0,12121.0,9055.0,0.8891,17926.0,1518.0
241,NaN,NaN,NaN,Phoenix,grandcanyonuniversity,85017,43.0,NaN,NaN,NaN,...,0.6716,31051.0,17050.0,17050.0,10977.0,1997.0,6482.0,0.0709,20570.0,1775.0
251,NaN,NaN,NaN,Chicago,nationallouisuniversity,60603,81.0,NaN,NaN,NaN,...,0.3083,24158.0,10440.0,10440.0,15427.0,6151.0,6148.0,0.9866,15579.0,283.0


In [175]:
#The data from collegescorecard is more accurate. 
#We use the kaggle data to fill in some of the missing sat average, 
#but otherwise we want to use the SAT average from the college scorecard dataset
data['sat_avg'] = data['SAT_AVG'].fillna(data['sat-avg'])

In [178]:
data['sat_avg'].isna().sum()

2

In [179]:
data.drop(['SAT_AVG','sat-avg'], axis=1, inplace=True)

In [187]:
#With the same principle, we will use enrollment data from college scorecard instead of Kaggle
data.drop(['enrollment'], axis=1, inplace=True)

In [188]:
data = data.rename(columns = {
    'act-avg': 'act_avg',
    'acceptance-rate': 'acceptance_rate',
    'percent-receiving-aid': 'percent_receiving_aid',
    'cost-after-aid': 'cost_after_aid',
    'hs-gpa-avg': 'hs_gpa_avg', 
    'INSTNM': 'institution_name',
    'NUMBRANCH': 'branches', 
    'REGION': 'region',
    'ADM_RATE': 'admission_rate',
    'SATVR25': 'satCR25', 
    'SATVR75': 'satCR75',
    'SATMT25': 'satmt25',
    'SATMT75': 'satmt75',
    'SATVRMID': 'satcrmid', 
    'SATMTMID': 'satmtmid',
    'ACTCM25': 'actcm25',
    'ACTCM75': 'actcm75',
    'ACTEN25': 'acten25', 
    'ACTEN75': 'acten75',
    'ACTMT25': 'actmt25',
    'ACTMT75': 'actmt75',
    'ACTCMMID': 'actcmmid', 
    'ACTENMID': 'actenmid',
    'ACTMTMID': 'actmtmid',
    'UGDS': 'enrollment', 
    'UGDS_WHITE': 'percent_white',
    'UGDS_BLACK': 'percent_black',
    'UGDS_HISP': 'percent_hispanic',
    'UGDS_ASIAN': 'percent_asian', 
    'UGDS_AIAN': 'percent_aian',
    'UGDS_NHPI': 'percent_nhpi', 
    'UGDS_2MOR': 'percent_twoormore',
    'UGDS_NRA': 'percent_nra',
    'UGDS_UNKN': 'percent_unknown', 
    'PPTUG_EF': 'percent_parttime',
    'COSTT4_A': 'avg_cost',
    'TUITIONFEE_IN': 'instante_tuition', 
    'TUITIONFEE_OUT': 'outstate_tuition',
    'TUITFTE': 'tuition_revenue_per', 
    'INEXPFTE': 'instructional_expenditure_per', 
    'AVGFACSAL': 'avg_faculty_salary', 
    'PFTFAC': 'ft_faculty_rate', 
    'NPT4': 'avg_net_price', 
    'NUM4': 'number_titleIV'
}
                  )

In [189]:
data.head()

,act_avg,city,sortName,zip,acceptance_rate,rankingDisplayScore,percent_receiving_aid,cost_after_aid,state,rankingSortRank,...,avg_cost,instante_tuition,outstate_tuition,tuition_revenue_per,instructional_expenditure_per,avg_faculty_salary,ft_faculty_rate,avg_net_price,number_titleIV,sat_avg
0,32.0,Princeton,princetonuniversity,8544,7.0,100.0,60.0,16793.0,NJ,1,...,64390.0,47140.0,47140.0,13216.0,53658.0,17519.0,0.8490,10027.0,346.0,1500.0
1,32.0,Cambridge,harvarduniversity,2138,5.0,98.0,55.0,16338.0,MA,2,...,66900.0,48949.0,48949.0,31930.0,43823.0,19741.0,0.8589,14327.0,350.0,1523.0
2,32.0,Chicago,universityofchicago,60637,8.0,96.0,42.0,27767.0,IL,3,...,72717.0,56034.0,56034.0,28508.0,89151.0,18115.0,0.8239,25455.0,494.0,1524.0
3,32.0,New Haven,yaleuniversity,6520,6.0,96.0,50.0,18385.0,CT,3,...,68950.0,51400.0,51400.0,17934.0,120956.0,17517.0,0.7459,18627.0,387.0,1520.0
4,32.0,New York,columbiauniversity,10027,6.0,95.0,48.0,21041.0,NY,5,...,71972.0,57208.0,57208.0,37294.0,89742.0,18713.0,0.4688,24231.0,526.0,1522.0


In [193]:
data.acceptance_rate.isna().sum()

0

In [194]:
data.admission_rate.isna().sum()

1

In [195]:
missing = data['admission_rate'].isna()
na_rows = data[missing]
na_rows

,act_avg,city,sortName,zip,acceptance_rate,rankingDisplayScore,percent_receiving_aid,cost_after_aid,state,rankingSortRank,...,avg_cost,instante_tuition,outstate_tuition,tuition_revenue_per,instructional_expenditure_per,avg_faculty_salary,ft_faculty_rate,avg_net_price,number_titleIV,sat_avg
263,16.0,Nashville,tennesseestateuniversity,37209,53.0,NaN,NaN,NaN,TN,-1,...,19058.0,7776.0,21132.0,6877.0,8732.0,7310.0,0.9707,11083.0,609.0,788.0


In [196]:
data.loc[263, 'acceptance_rate']

53.0

In [197]:
data.at[263, 'admission_rate'] = 0.53

In [198]:
data.drop(['acceptance_rate'], axis=1, inplace=True)

In [199]:
data.to_csv('cleaned_data.csv', index=False)

In [200]:
data.shape

(300, 62)